In [1]:
from challenge import agoda_cancellation_prediction
import sys

import pandas as pd

sys.path.append("../")
from utils import *
from agoda_cancellation_estimator import *
import agoda_cancellation_preprocessor
from agoda_cancellation_prediction import *
from agoda_cancellation_preprocessor import AgodaCancellationPreprocessor
from sklearn.metrics import roc_curve, auc, f1_score, recall_score,\
	precision_score, classification_report, confusion_matrix
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier,\
	GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV, RidgeClassifier
from sklearn.model_selection import GridSearchCV

import importlib

importlib.reload(agoda_cancellation_preprocessor)
from agoda_cancellation_preprocessor import *

importlib.reload(agoda_cancellation_prediction)
from agoda_cancellation_prediction import *

np.random.seed(0)
c = [custom[0], custom[-1]]

# Load and preprocess data
full_data = load_data(
	"../datasets/agoda_cancellation_train.csv")
p = AgodaCancellationPreprocessor(full_data)
base_design_matrix, week_specific = p.preprocess(full_data)
cancellation_labels_list = p.preprocess_labels(full_data.cancellation_datetime,
                                               full_data.booking_datetime)
design_matrix = pd.DataFrame()
cancellation_labels = pd.DataFrame()
for i in range(len(week_specific)):
	pd.concat([design_matrix,
	           pd.concat([base_design_matrix, week_specific[i]], axis=1)])
	pd.concat([cancellation_labels, cancellation_labels_list[i]])
for i in range(1, WEEK - 3):
	week_data = pd.read_csv(f"week_{i}_test_data.csv")
	test_set_i = p.preprocess(week_data)[0]
	test_set_i_labels = pd.read_csv(f"week_{i}_labels.csv")[
		"cancel"].astype(int)
	p.add_week_features(test_set_i, week_data)
	design_matrix = pd.concat(
		[design_matrix, test_set_i])
	cancellation_labels = pd.concat(
		[cancellation_labels, test_set_i_labels])
	test_set_i = fill_missing_columns(design_matrix, test_set_i)

week_data = pd.read_csv(f"week_{WEEK - 2}_test_data.csv")
train_test_set = p.preprocess(week_data)[0]
train_test_set_labels = pd.read_csv(f"week_{WEEK - 2}_labels.csv")[
	"cancel"].astype(int)
p.add_week_features(train_test_set, week_data)
train_test_set = fill_missing_columns(design_matrix, train_test_set)

week_data = pd.read_csv(f"week_{WEEK - 1}_test_data.csv")
test_set = p.preprocess(week_data)[0]
test_set_labels = pd.read_csv(f"week_{WEEK - 2}_labels.csv")[
	"cancel"].astype(int)
p.add_week_features(test_set, week_data)
test_set = fill_missing_columns(design_matrix, test_set)


In [3]:
import agoda_cancellation_estimator
import importlib

importlib.reload(agoda_cancellation_estimator)
from agoda_cancellation_estimator import *

train_X = design_matrix
train_y = np.array(cancellation_labels).reshape((-1,))
test_X = test_set
test_y = np.array(test_set_labels).reshape((-1,))
train_test_X = train_test_set
train_test_y = train_test_set_labels

# Fit model over data with different model seeds.
scores1 = []
scores2 = []

param_grid = {'max_leaf_nodes': range(20, 100, 10),
              'n_estimators': range(100, 300, 50), 'max_depth': range(5, 15)}

random_forest_cv = GridSearchCV(
	RandomForestClassifier(random_state=1, class_weight='balanced_subsample',
	                       n_jobs=8),
	cv=5, param_grid=param_grid,
	scoring=lambda label, pred: f1_score(label, pred, labels=[0, 1],
	                                     average="macro")).fit(train_X,train_y)
cv_errors = 1 - random_forest_cv.cv_results_["mean_test_score"]
std = random_forest_cv.cv_results_["std_test_score"]

minimizer = np.ravel_multi_index(np.argmin(cv_errors),cv_errors.shape)
print(cv_errors[minimizer])

# for n in range(1, 20):
# 	model = RandomForestClassifier(random_state=1, max_leaf_nodes=40,
# 	                               n_estimators=200,
# 	                               max_depth=10,
# 	                               class_weight='balanced_subsample', n_jobs=8)
# 	# model=GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_features='auto', max_depth=5,
# 	#                                  )
# 	model.fit(train_X, train_y)
# 	y_pred = model.predict(test_X)
# 	f1 = f1_score(test_y, y_pred, labels=[0, 1], average="macro")
# 	precision = precision_score(test_y, y_pred, labels=[0, 1],
# 	                            average="macro")
# 	recall = recall_score(test_y, y_pred, labels=[0, 1], average="macro")
# 	print(f"random seed: {n}")
# 	print(
# 		f"f1 score: {f1}, precision score: {precision}, recall score: {recall}.")
# 	print(confusion_matrix(test_y, y_pred, labels=[0, 1]))
# 	scores1.append(f1)
# 	y_pred = model.predict(train_test_X)
# 	f1 = f1_score(train_test_y, y_pred, labels=[0, 1], average="macro")
# 	precision = precision_score(train_test_y, y_pred, labels=[0, 1],
# 	                            average="macro")
# 	recall = recall_score(train_test_y, y_pred, labels=[0, 1], average="macro")
# 	print(
# 		f"f1 score: {f1}, precision score: {precision}, recall score: {recall}.")
# 	print(confusion_matrix(train_test_y, y_pred, labels=[0, 1]))
# 	print()
# 	scores2.append(f1)
#
# # scores = pd.DataFrame([scores1])
# # px.line(scores,x=scores[:,0]).show()
# fig = go.Figure(
# 	[go.Scatter(x=list(range(1, 20)), y=scores1, mode="lines+markers")],
# 	[go.Scatter(x=list(range(1, 20)), y=scores2, mode="lines+markers")])
# fig.update_layout(go.Layout(margin=dict(t=100)))
# fig.show()





c:\users\0erel\onedrive\documents\cs.bsc\semd\iml\iml.huji repo\venv\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1600 fits failed out of a total of 1600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1600 fits failed with the following error:
Traceback (most recent call last):
  File "c:\users\0erel\onedrive\documents\cs.bsc\semd\iml\iml.huji repo\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\0erel\onedrive\documents\cs.bsc\semd\iml\iml.huji repo\venv\lib\site-packages\sklearn\ensemble\_forest.py", line 328, in fit
    X, y, multi_output=True, accept_sparse="csc", dtype=DTYPE
  File "c:\u

NotFittedError: All estimators failed to fit